In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Reshape
from tensorflow.keras.optimizers import RMSprop, Adam

from matplotlib import pyplot as plt
import numpy as np

In [ ]:
class PDESolver(Sequential):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.mse = tf.keras.losses.MeanSquaredError()

    @property
    def metrics(self):
      return [self.loss_tracker]

    def train_step(self, data):
        #batch_size=tf.shape(data)[0]
        batch_size=10
        x = tf.random.uniform((batch_size,1), minval=-5 maxval=5

        with tf.GradientTape() as tape:
            with tf.GradientTape(persistent=True) as g:
                g.watch(x)
                with tf.GradientTape() as gg:
                    gg.watch(x)
                    input = tf.concat((x,t),axis=1)
                    y_pred = self(input, training=True)
                y_x=gg.gradient(y_pred,x)
            y_xx=g.gradient(y_x,x)
            
            eq = y_xx + y_x
            ci1 = 1.0
            ci2 = -0.5

            t_init = tf.zeros(x.shape)
            input_ini = tf.concat((x,t_init),axis=1)
            y_init = self(input_ini, training=True)
            loss = self.mse(0., eq + self.mse(tf.math.sin(x),y_init)

        # Compute grad
        grads = tape.gradient(loss, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        # Compute our own metrics
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

In [ ]:
model = PDESolver()
model.add(Dense(100, activation='tanh', input_shape=(2,), name="primera"))
model.add(Dense(10, activation='tanh'))
model.add(Dense(1, activation='linear'))
#model.add(Reshape(()))
model.summary()

In [ ]:
model.compile(optimizer="adam")

In [ ]:
x = tf.random.uniform((10,1), minval=0, maxval=10)
t = tf.random.uniform((10,1),minval=0,maxval=15)
input = tf.concat((x,t),axis=1)
history = model.fit(input,epochs=500,verbose=0)
plt.plot(history.history["loss"])